In [45]:
import datetime as dt
import os
from pathlib import Path

import basketball_reference_web_scraper
from basketball_reference_web_scraper import client
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


# Constants to translate raw NBA stats to DraftKings Points.
# Pulling them out here in case they change it's easy to swap out
POINTS_MULTIPLE = 1
THREE_POINT_MULTIPLE = 0.5
REBOUND_MULTIPLE = 1.25
ASSIST_MULTIPLE = 1.5
STEAL_MULTIPLE = 2
BLOCK_MULTIPLE = 2
TURNOVER_MULTIPLE = -0.5
DOUBLE_DOUBLE_MULTIPLE = 1.5
TRIPLE_DOUBLE_MULTIPLE = 3


In [74]:
# https://jaebradley.github.io/basketball_reference_web_scraper/api/
player_box_scores = client.player_box_scores(day=7, month=8, year=2020)

In [47]:
type(player_box_scores)

list

In [48]:
len(player_box_scores)

130

In [49]:
type(player_box_scores[0])

dict

In [50]:
player_box_scores[0]

{'assists': 12,
 'attempted_field_goals': 21,
 'attempted_free_throws': 10,
 'attempted_three_point_field_goals': 18,
 'blocks': 0,
 'defensive_rebounds': 4,
 'game_score': 44.5,
 'location': <Location.AWAY: 'AWAY'>,
 'made_field_goals': 13,
 'made_free_throws': 8,
 'made_three_point_field_goals': 11,
 'name': 'Damian Lillard',
 'offensive_rebounds': 0,
 'opponent': <Team.DENVER_NUGGETS: 'DENVER NUGGETS'>,
 'outcome': <Outcome.WIN: 'WIN'>,
 'personal_fouls': 2,
 'seconds_played': 2444,
 'slug': 'lillada01',
 'steals': 3,
 'team': <Team.PORTLAND_TRAIL_BLAZERS: 'PORTLAND TRAIL BLAZERS'>,
 'turnovers': 2}

In [78]:
player_box_scores_df = pd.DataFrame(player_box_scores)
player_box_scores_df['date_played'] = dt.date(2020, 8, 7)
player_box_scores_df['total_rebounds'] = player_box_scores_df['offensive_rebounds'] + player_box_scores_df['defensive_rebounds']
player_box_scores_df['total_points_scored'] = (player_box_scores_df['made_field_goals'] - player_box_scores_df['made_three_point_field_goals']) * 2 \
    + player_box_scores_df['made_three_point_field_goals'] * 3 \
    + player_box_scores_df['made_free_throws']
player_box_scores_df['total_double_digit_stats'] = (player_box_scores_df['total_points_scored'] >= 10).astype(int) \
    + (player_box_scores_df['total_rebounds'] >= 10).astype(int) \
    + (player_box_scores_df['assists'] >= 10).astype(int) \
    + (player_box_scores_df['steals'] >= 10).astype(int) \
    + (player_box_scores_df['blocks'] >= 10).astype(int)
player_box_scores_df['double_double_flag'] = (player_box_scores_df['total_double_digit_stats'] == 2).astype(int)
player_box_scores_df['triple_double_flag'] = (player_box_scores_df['total_double_digit_stats'] >= 3).astype(int)

In [79]:
player_box_scores_df['draftkings_points'] = \
    POINTS_MULTIPLE * player_box_scores_df['total_points_scored'] \
    + THREE_POINT_MULTIPLE * player_box_scores_df['made_three_point_field_goals'] \
    + REBOUND_MULTIPLE * player_box_scores_df['total_rebounds'] \
    + ASSIST_MULTIPLE * player_box_scores_df['assists'] \
    + STEAL_MULTIPLE * player_box_scores_df['steals'] \
    + BLOCK_MULTIPLE * player_box_scores_df['blocks'] \
    + TURNOVER_MULTIPLE * player_box_scores_df['turnovers'] \
    + DOUBLE_DOUBLE_MULTIPLE * player_box_scores_df['double_double_flag'] \
    + TRIPLE_DOUBLE_MULTIPLE * player_box_scores_df['triple_double_flag']

In [80]:
player_box_scores_df = player_box_scores_df[[
    'slug',
    'name',
    'team',
    'location',
    'opponent',
    'date_played',
    'outcome',
    'seconds_played',
    'made_field_goals',
    'attempted_field_goals',
    'made_three_point_field_goals',
    'attempted_three_point_field_goals',
    'made_free_throws',
    'attempted_free_throws',
    'total_points_scored',
    'total_rebounds',
    'assists',
    'steals',
    'blocks',
    'turnovers',
    'double_double_flag',
    'triple_double_flag',
    'draftkings_points',    
]]

In [81]:
player_box_scores_df.sort_values('draftkings_points', ascending=False)

,slug,name,team,location,opponent,date_played,outcome,seconds_played,made_field_goals,attempted_field_goals,made_three_point_field_goals,attempted_three_point_field_goals,made_free_throws,attempted_free_throws,total_points_scored,total_rebounds,assists,steals,blocks,turnovers,double_double_flag,triple_double_flag,draftkings_points
13,smithis01,Ish Smith,Team.WASHINGTON_WIZARDS,Location.AWAY,Team.NEW_ORLEANS_PELICANS,2020-08-07,Outcome.LOSS,1909,7,16,2,4,2,2,18,6,10,4,0,3,1,0,49.50
15,harrito02,Tobias Harris,Team.PHILADELPHIA_76ERS,Location.HOME,Team.ORLANDO_MAGIC,2020-08-07,Outcome.WIN,2383,8,21,2,5,5,7,23,15,4,0,0,2,1,0,49.25
7,browntr01,Troy Brown,Team.WASHINGTON_WIZARDS,Location.AWAY,Team.NEW_ORLEANS_PELICANS,2020-08-07,Outcome.LOSS,2310,6,16,3,5,5,5,20,10,5,2,0,1,1,0,46.50
3,allenja01,Jarrett Allen,Team.BROOKLYN_NETS,Location.HOME,Team.SACRAMENTO_KINGS,2020-08-07,Outcome.WIN,2188,5,8,0,0,7,10,17,11,8,0,1,1,1,0,45.75
17,leverca01,Caris LeVert,Team.BROOKLYN_NETS,Location.HOME,Team.SACRAMENTO_KINGS,2020-08-07,Outcome.WIN,2166,9,21,3,8,1,4,22,5,7,3,0,2,0,0,45.25
14,vucevni01,Nikola Vučević,Team.ORLANDO_MAGIC,Location.AWAY,Team.PHILADELPHIA_76ERS,2020-08-07,Outcome.LOSS,2080,9,18,2,6,1,2,21,12,3,1,0,1,1,0,44.50
0,holidjr01,Jrue Holiday,Team.NEW_ORLEANS_PELICANS,Location.HOME,Team.WASHINGTON_WIZARDS,2020-08-07,Outcome.WIN,2197,11,16,2,4,4,5,28,3,6,2,1,7,0,0,44.25
1,whitede01,Derrick White,Team.SAN_ANTONIO_SPURS,Location.HOME,Team.UTAH_JAZZ,2020-08-07,Outcome.WIN,1982,8,15,4,9,4,5,24,6,4,1,1,1,0,0,43.00
6,bryanth01,Thomas Bryant,Team.WASHINGTON_WIZARDS,Location.AWAY,Team.NEW_ORLEANS_PELICANS,2020-08-07,Outcome.LOSS,2093,9,18,2,7,2,2,22,8,1,2,2,1,0,0,42.00
4,bradlto01,Tony Bradley,Team.UTAH_JAZZ,Location.AWAY,Team.SAN_ANTONIO_SPURS,2020-08-07,Outcome.LOSS,1329,7,9,1,1,0,1,15,11,2,1,3,0,1,0,41.75


In [82]:
ROOT_DIR = os.path.abspath(os.curdir)
ROOT_DIR = str(Path(ROOT_DIR).parents[0])
ROOT_DIR

'C:\\Users\\Kevin\\Documents\\GitHub\\draftkings'

In [83]:
player_box_scores_df.to_csv(ROOT_DIR + "\\" + r'data\raw\nba_box_score_stats\nba_box_score_stats_20200807.csv', index=False, encoding='utf-8')

AttributeError: 'DataFrame' object has no attribute 'encode'